In [1]:
import pandas as pd
from dateutil.parser import parse
from datetime import datetime
from datetime import timedelta

key_words = ["U.S.-China trade and COVID-19","Unfair trade and intellectual property rights","U.S.-China trade and tariffs","U.S.-China trade war and trade barriers","U.S.-China trade war and protectionism","China and the world order","China and national Security","presidential election and trade war"]

key_words_number = len(key_words)

websites = ["Huffpost","the Washington Post","Fox News","Wall Street journal"]

websites_number = len(websites)

In [2]:
def show_number_info():
    for i in range(websites_number):
        print()
        print("Website:",websites[i])
        print()
        for j in range(key_words_number):
            file_path = "~/data/data_%s_%s.json"%(websites[i],key_words[j])
            data = pd.read_json(file_path)
            print("Keywords:%s Number: %s"%(key_words[j],len(data)))

In [3]:
def merge_data_website(website):
    pieces = []
    for i in range(key_words_number):
        file_path = "~/data/data_%s_%s.json"%(website,key_words[i])
        frame = pd.read_json(file_path)
        pieces.append(frame)
    data = pd.concat(pieces, ignore_index=True)
    return data

In [5]:
websites_merge_data = []
for website in websites:
    data = merge_data_website(website).drop_duplicates(['content'])
    websites_merge_data.append(data)

/home/congw/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [6]:
def time_transform(x):
    start = datetime(2020, 8, 1)
    if x.split()[-1]=="ago":
        days_ago = x.split()[0]
        date_str = start-timedelta(int(days_ago))
        return date_str
    else:
        return parse(x)

websites_merge_data[2].date = websites_merge_data[2].date.map(time_transform)

In [8]:
index = pd.date_range(start='20200701', periods=31, freq='D')

In [9]:
def count_week(website):
    return website.set_index('date').resample('W-SUN').count().url.reset_index()

count_weeks=[]
for i in range(websites_number):
    CW = count_week(websites_merge_data[i]).rename(columns={"url":"number"})
    count_weeks.append(CW)

def topic_count_week(website):
    return website.set_index('date').groupby('topic').resample('W-SUN').count().url.reset_index()

count_topic_weeks=[]
for i in range(websites_number):
    CTW = topic_count_week(websites_merge_data[i]).rename(columns={"url":"number"})
    count_topic_weeks.append(CTW)

# for i in range(websites_number):
#     count_weeks[i] = count_weeks[i].set_index('date')
#     count_topic_weeks[i] = count_topic_weeks[i].set_index('date')

import numpy as np

dn = pd.merge(pd.merge(pd.merge(count_weeks[0],count_weeks[1],on='date'),count_weeks[2],on='date'),count_weeks[3],on='date')

dn.columns = ["date"]+websites

In [51]:
date_range=['7.1-7.5','7.6-7.12','7.13-7.19','7.20-7.26','7.27-7.31']

dn = dn.set_axis(date_range)

dn = dn.drop('date',axis=1)

dn.plot(figsize=(10,8))

In [87]:
def change_format(df):
    return df.set_index(['date','topic']).unstack().fillna(0)['number']

change_format(count_topic_weeks[0]).to_excel("e.xlsx")

for i,df in enumerate(count_topic_weeks):
    change_format(df).to_excel("Topic-%s.xlsx"%websites[i])

In [ ]:
from wordcloud import WordCloud

f = open(u'txt/AliceEN.txt','r').read()
wordcloud = WordCloud(background_color="white",width=1000, height=860, margin=2).generate(f)

# width,height,margin可以设置图片属性

# generate 可以对全部文本进行自动分词,但是他对中文支持不好,对中文的分词处理请看我的下一篇文章
#wordcloud = WordCloud(font_path = r'D:\Fonts\simkai.ttf').generate(f)
# 你可以通过font_path参数来设置字体集

#background_color参数为设置背景颜色,默认颜色为黑色

import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

wordcloud.to_file('test.png')
# 保存图片,但是在第三模块的例子中 图片大小将会按照 mask 保存